# ws 02 03

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN 

In [ ]:
pitch = 20
step =  1
N = 100
n_train = int(N * 0.7)   

def gen_data(x):
    return (x % pitch ) / pitch 


t = np.arange(1, N+1) 

y = [gen_data(i) for i in t]

y = np.array(y)

print(f' max: {np.max(y)}')
print(y[:15])

In [ ]:
plt.figure(figsize=(7,3.2))
plt.plot(y)
plt.show()

In [ ]:
# convert into dataset matrix
def convertToMatrix(data, step=1):  
    X, Y = [], []
    for i in range(len(data)-step):
        d = i + step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

In [ ]:

train, test = y[0:n_train], y[n_train:N]

print('train test (before conversion):', train.shape, test.shape)
 
X_train, y_train = convertToMatrix(train, step) 
X_test, y_test = convertToMatrix(test, step)

print('train test (after conversion):', X_train.shape, X_test.shape)

In [ ]:
print(X_train[:7])

In [ ]:
print(y_train[:6])

In [ ]:

X_train = np.expand_dims(X_train, -1) 
X_test = np.expand_dims(X_test, -1) 

X_train.shape, y_train.shape

In [ ]:
X_train[:5]


In [ ]:
y_train[:10]

In [ ]:
y_train.shape

## Model

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(step, 1), activation="relu"))

model.add(Dense(1))

model.compile(loss='mean_squared_error')
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, 'rnn2.png', show_shapes=True, show_layer_names=False)

In [ ]:

history = model.fit(X_train, y_train, epochs=20+10, batch_size=1, verbose=0)

In [ ]:
plt.plot(history.history['loss'])
plt.show()

## Eval

In [ ]:
trainScore = model.evaluate(X_train, y_train, verbose=0)
trainScore

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

y_pred_train = np.squeeze(trainPredict)
print('Y train shape:', y_train.shape, y_pred_train.shape)

y_pred_test = np.squeeze(testPredict)
print('Y test shape:',y_test.shape, y_pred_test.shape)

r2_train = r2_score(y_train, y_pred_train)  
r2_test = r2_score(y_test, y_pred_test)

print('R2 Train: {:.3f}'. format(r2_train))
print('R2 Test: {:.3f}'. format(r2_test))

In [ ]:
prefix = np.empty(step).reshape(-1, 1)
prefix.fill(np.nan)

trainPredict = np.concatenate((prefix, trainPredict), axis=0) 
testPredict = np.concatenate((prefix, testPredict), axis=0) 

predicted = np.concatenate((trainPredict,testPredict),axis=0)


In [ ]:

plt.figure(figsize=(9,3.5))

plt.plot(y * pitch , lw=1.2, label='Dataset')  # dataset
plt.plot(predicted * pitch , 'r--', lw=2.5, label='Prediction')
plt.axvline(n_train, c='m', lw=2.5)

plt.legend(loc=2)
plt.show()

In [ ]:
new_x = 1
new_x = np.array(new_x).reshape(1, 1, 1)
pred = model.predict(new_x / pitch) 
prediction = pred * pitch
print(pred)

In [ ]:
new_x/pitch

In [ ]:
X_test

In [ ]:
y_test[:10] * pitch

In [ ]:
# pred = model.predict(X_train) * pitch
pred = model.predict(X_test) * pitch
np.squeeze(pred)[:10].round(1)